# Práctico 2c: Neurona de Izhikevich

## Configuración

Ejecutá las siguientes celdas para configurar el entorno del notebook

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import ipywidgets as widgets

### Funciones utilitarias

In [ ]:
def I_inj(t, amp):
  """Crea una función de corriente inyectada con un pulsos cuadrado.

  Args:
    t (ndarray): vector 1D de tiempos (ms), estrictamente creciente.
    amp (float): amplitud del pulso, en uA/cm^2.

  Returns:
    function: una función I(t) que devuelve la corriente inyectada I(t).
  """
  tmax = np.max(t)
  inicios = np.linspace(0, tmax, 8)
  inicio = inicios[1]
  fin = inicios[6]
  
  def I(t):
    return amp * (t > inicio) - amp * (t > fin)

  return I

### Funciones de graficado

In [ ]:
def plot_izikhevich(t, I, V, u):
  """Grafica la dinámica del modelo de Izikhevich.

  Args:
    t (ndarray): vector 1D de tiempos (ms), estrictamente creciente y con paso uniforme.
    I (function): función escalar I(t) que devuelve la corriente externa (mA) en cada t.
    V (ndarray): potencial de membrana.
    u (ndarray): variable de recobro.
  """
  fig, [ax1, ax2, ax3] = plt.subplots(3, 1, figsize=(8, 6), gridspec_kw={"height_ratios": [3, 2, 1]}, sharex=True)

  ax1.plot(t, V)
  ax1.set_ylim(-100, 50)
  ax1.set_ylabel("V (mV)")
  ax1.grid(True)
  
  ax2.plot(t, u)
  ax2.set_ylim(-20, 20)
  ax2.set_ylabel("u")
  ax2.grid(True)
  
  ax3.plot(t, I(t))
  ax3.set_ylim(-20, 20)  
  ax3.set_ylabel("I (mA)")
  ax3.set_xlabel("t (ms)")
  
  plt.show()

## Definición del modelo

Izhikevich (2003) propuso un modelo simple de neurona que se describe con dos ecuaciones diferenciales acopladas. Una para el potencial de membrana $v$ y otra para el valor de recobro $u$:

$$\dfrac{dv(t)}{dt} = 0.04 v^2(t) + 5 v(t) + 140 - u + I(t)$$
$$\dfrac{du(t)}{dt} = a(bv - u)$$

En conjunto con las condiciones de reestablecimiento:

$$v(v > \theta) = c$$
$$u(v > \theta) = u + d$$

Este modelo cumple con el requisito de ser computacionalmente eficiente y además de producir un variado abanico de patrones de disparos observados en neuronas corticales. 

### Rol de los parámetros en el modelo de Izhikevich

El modelo de Izhikevich utiliza cuatro parámetros principales (a, b, c, d) para modelar una amplia variedad de comportamientos neuronales.

Aquí se describe el rol de cada parámetro:

* **`a`**: Controla la **velocidad de recuperación** de la variable de adaptación `u`. Valores bajos de `a`  hacen que la neurona se recupere lentamente después de un pico, mientras que valores altos producen una recuperación rápida. Esto influye en la frecuencia de disparo de la neurona.

* **`b`**:  Describe la **sensibilidad** de la variable de adaptación `u` al potencial de membrana `v`.  Valores altos de `b` hacen que la neurona sea más sensible a los cambios en `v`, lo que puede producir comportamientos como el rebote post-inhibitorio.

* **`c`**: Representa el **potencial de reinicio** de la membrana después de un pico.  Este valor determina a qué voltaje se restablece la membrana después de que la neurona ha disparado.

* **`d`**:  Controla la **recuperación de la variable de adaptación `u`** después de un pico.  Un valor alto de `d`  provoca un aumento rápido en `u` después de un pico, lo que puede influir en la duración del período refractario y la frecuencia de disparo.

En resumen, los parámetros `a`, `b`, `c` y `d`  permiten ajustar la dinámica del modelo de Izhikevich para reproducir una amplia gama de comportamientos neuronales, como disparos tónicos, ráfagas, rebotes post-inhibitorios, y más.

Ajustar estos parámetros de manera precisa es crucial para simular neuronas específicas y para estudiar la dinámica de las redes neuronales.

### Constantes

In [ ]:
# Condición inicial de la variable de voltaje v(t) en t=0, es decir, valor de v(0), en mV.
v0 = -75

# Umbral de voltaje para el recobro, en mV
threshold = 30

### Integración

In [ ]:
def vdot(v, u, I):
  return (0.04 * v + 5) * v + 140 - u + I

def udot(u, v, a, b):
  return a * (b * v - u)
  
def integrate_izikhevich(t, I, a=0.02, b=0.2, c=-65, d=2):
  # Condición inicial de la variable de recobro u(t) en t=0, es decir, valor de v(0).
  u0 = v0 * b
  
  # Variable de voltaje
  v = [v0]
  
  # Variable de recobro
  u = [u0]

  # Calculamos el valor de salto en el tiempo
  dt = t[1] - t[0]
  
  # Integramos con el método de Euler
  for step in t[:-1]:
    if v[-1] < threshold:
      v = np.append(v, v[-1] + dt * vdot(v[-1], u[-1], I(step)))
      u = np.append(u, u[-1] + dt * udot(u[-1], v[-1], a, b))
    else:
      v[-1] = threshold
      v = np.append(v, c)
      u = np.append(u, u[-1] + d)
  return v, u

## Simulación del modelo

Veamos como hacerlo en Python integrando estas ecuaciones diferenciales utilizando el método de Euler segun vimos en el Cuaderno 3:

In [ ]:
# I_iny: Amplitud de la corriente inyectada, en mA
# a, b, c, d: Parametros del modelo de Izhikhevich
@widgets.interact(amp=(-10, 10, 0.1), a=(0, 0.2, 0.01), b=(0, 0.2, 0.01), c=(-75, -50, 0.1), d=(0, 10, 0.1))
def simulate(amp=10, a=0.02, b=0.2, c=-65, d=2):
  t = np.arange(0, 200, 0.05)
  I = I_inj(t, amp)
  V, u = integrate_izikhevich(t, I, a, b, c, d)
  plot_izikhevich(t, I, V, u)

## Simulaciones de la neurona de Izhikevich

Veamos que sucede cuando inyectamos una corriente constante de $I_{iny}=10mA$ durante $200{ms}$ usando los siguientes parámetros:

$\begin{split} a &= 0.02 \\ b &= 0.2 \\ c &= -65 \\ d &= 2 \end{split}$

*Nota: Siempre integramos con $\Delta t=0.5$*

In [ ]:
amp = 10
t = np.arange(0, 200, 0.05)
I = I_inj(t, amp)
V, u = integrate_izikhevich(t, I, a=0.02, b=0.2, c=-65, d=2)
plot_izikhevich(t, I, V, u)

###  Neuronas FS

El parámetro $a$ establece la escala temporal de la variable de recobro. Valores pequeños de $a$ llevan a recuperaciones lentas. Veamos que sucede cuando subimos el valor de $a$ sin cambiar los demás parámetros:

$a = 0.1$

In [ ]:
amp = 10
t = np.arange(0, 200, 0.05)
I = I_inj(t, amp)
V, u = integrate_izikhevich(t, I, a=0.1)
plot_izikhevich(t, I, V, u)

### Neuronas RS

El valor $d$ describe el reestablecimiento de la variable de recobro.  Veamos que sucede cuando subimos el valor de $d$ sin cambiar los demás parámetros:

$d=8$

In [ ]:
amp = 10
t = np.arange(0, 200, 0.05)
I = I_inj(t, amp)
V, u = integrate_izikhevich(t, I, d=8)
plot_izikhevich(t, I, V, u)

### Neuronas CH

El valor $c$ establece el voltaje al que se reestablece la membrana luego de un disparo.  Veamos que sucede cuando subimos el valor de $c$ sin cambiar los demás parámetros:

$c=-50$

In [ ]:
amp = 10
t = np.arange(0, 200, 0.05)
I = I_inj(t, amp)
V, u = integrate_izikhevich(t, I, c=-50)
plot_izikhevich(t, I, V, u)

## Nulclinas

Para encontrar las nulclinas del modelo de Izhikevich:

$\begin{split} 
0 &= 0.04v^2+5v+140-u+I \\
0 &= a(bv-u)
\end{split}$

In [ ]:
amp = 10 
a = 0.2 
b = 0.2
c = -20 
d = -8

v = np.arange(-100, 0, 0.5)
plt.plot(v, 0.04 * v**2 + 5 * v + 140 + amp)
plt.plot(v, b * v)
plt.show()

## Referencias

Izhikevich, E. M. (2003). Simple model of spiking neurons. *IEEE Transactions on Neural Networks*, 14(6), 1569–1572. [doi:10.1109/tnn.2003.820440](https://doi.org/10.1109/TNN.2003.820440)